In [1]:
# 📘 07_app.ipynb — Application Gradio finale

import gradio as gr
import pandas as pd
import os
from predict import predict_price, predict_tranche  # doit être dans le même dossier ou accessible dans PYTHONPATH

# 📂 Emplacement du fichier de log
LOG_PATH = "flagged/log.csv"

# 🔸 Niveaux prédéfinis de fiabilité (%)
choices = {
    "Acceptable": 80,
    "Moyenne": 85,
    "Bonne": 90,
    "Très Bonne": 96,
    "Excellente": 99
}

# 🔍 Fonction principale de prédiction
def faire_une_prediction(description, niveau, use_predefined, fiabilite_percent, fiabilite_choix):
    fiabilite = (choices[fiabilite_choix] if use_predefined else fiabilite_percent) / 100
    try:
        prix = predict_price(description, fiabilite)
        tranche = predict_tranche(description, fiabilite)
        return round(prix, 2), tranche
    except Exception as e:
        return f"Erreur : {str(e)}", ""

# 📝 Fonction d'enregistrement dans le fichier log
def enregistrer_log(description, niveau, use_predefined, fiabilite_percent, fiabilite_choix, prix, tranche):
    fiabilite = choices[fiabilite_choix] if use_predefined else fiabilite_percent
    log_data = {
        "Description": description,
        "Niveau": niveau,
        "Fiabilité (%)": fiabilite,
        "Prix prédit (€)": prix,
        "Tranche prédite": tranche
    }

    df_log = pd.DataFrame([log_data])
    os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)
    if os.path.exists(LOG_PATH):
        df_log.to_csv(LOG_PATH, mode="a", index=False, header=False)
    else:
        df_log.to_csv(LOG_PATH, index=False)

    return "✅ Signalement enregistré avec succès."

# 🖥️ Interface utilisateur Gradio
with gr.Blocks() as iface:
    gr.Markdown("""
    ## 🎯 Application de prédiction de prix Fiverr
    Cette application permet d’estimer automatiquement :
    - 💰 Le **prix probable** d’un service publié sur Fiverr,
    - 📊 Sa **tranche de prix** parmi trois catégories (Basse / Moyenne / Haute).
    Elle s’appuie sur un pipeline hybride combinant des embeddings de description et des variables numériques.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            description = gr.Textbox(
                label="✏️ Titre du service",
                placeholder="Ex : Je fais le ménage",
                value="Je fais le ménage"  # Valeur par défaut
            )
            niveau =  gr.Dropdown(label="🔰 Niveau du vendeur", choices=["Beginner", "Intermediate", "Expert"], value="Beginner", visible=False)
            use_predefined = gr.Checkbox(label="🎛️ Utiliser les niveaux prédéfinis de fiabilité", value=True)

            fiabilite_percent = gr.Slider(label="Fiabilité (%)", minimum=0, maximum=100, value=80, step=5, visible=False)
            fiabilite_choix = gr.Radio(
                label="🎚️ Choisissez un niveau de fiabilité",
                choices=list(choices.keys()),
                value="Bonne",
                visible=True
            )

            def sync_slider_with_radio(choix):
                return gr.update(value=choices[choix])

            fiabilite_choix.change(sync_slider_with_radio, inputs=fiabilite_choix, outputs=fiabilite_percent)

            def toggle_inputs(use_predef):
                return {
                    fiabilite_percent: gr.update(visible=not use_predef),
                    fiabilite_choix: gr.update(visible=use_predef)
                }

            use_predefined.change(toggle_inputs, inputs=use_predefined, outputs=[fiabilite_percent, fiabilite_choix])

            bouton_predire = gr.Button("📈 Estimer le prix")

        with gr.Column(scale=1):
            sortie_prix = gr.Textbox(label="💰 Prix estimé")
            sortie_tranche = gr.Textbox(label="📊 Tranche estimée")
            bouton_signaler = gr.Button("🚨 Ajouter au fichier log.csv")
            confirmation = gr.Textbox(label="✅ Confirmation", visible=False)

            bouton_predire.click(
                fn=faire_une_prediction,
                inputs=[description, niveau, use_predefined, fiabilite_percent, fiabilite_choix],
                outputs=[sortie_prix, sortie_tranche]
            )

            bouton_signaler.click(
                fn=enregistrer_log,
                inputs=[description, niveau, use_predefined, fiabilite_percent, fiabilite_choix, sortie_prix, sortie_tranche],
                outputs=confirmation
            )

# 🚀 Lancement
iface.launch()

c:\wamp64\www\ia.follaco.fr\.venv\lib\site-packages\gradio_client\documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
c:\wamp64\www\ia.follaco.fr\.venv\lib\site-packages\gradio_client\documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.39.0, however version 4.44.1 is available, please upgrade.
--------


In [2]:
# Exemple de description et niveau à tester
desc = "I will design a professional logo for your business"
# niveau = "Beginner" # "Beginner", "Intermediate", "Expert"

# Test de l’impact de la fiabilité sur la prédiction du prix
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]:
    prix = predict_price(desc, f)
    print(f"Fiabilité = {f*100:.0f}% → Prix prédit = {prix} €")

Fiabilité = 0% → Prix prédit = 14.2 €
Fiabilité = 10% → Prix prédit = 14.2 €
Fiabilité = 20% → Prix prédit = 14.2 €
Fiabilité = 30% → Prix prédit = 14.2 €
Fiabilité = 40% → Prix prédit = 14.2 €
Fiabilité = 50% → Prix prédit = 14.2 €
Fiabilité = 60% → Prix prédit = 14.2 €
Fiabilité = 70% → Prix prédit = 14.2 €
Fiabilité = 80% → Prix prédit = 14.2 €
Fiabilité = 81% → Prix prédit = 14.2 €
Fiabilité = 82% → Prix prédit = 14.2 €
Fiabilité = 83% → Prix prédit = 14.2 €
Fiabilité = 84% → Prix prédit = 14.6 €
Fiabilité = 85% → Prix prédit = 14.6 €
Fiabilité = 90% → Prix prédit = 15.28 €
Fiabilité = 91% → Prix prédit = 15.28 €
Fiabilité = 92% → Prix prédit = 15.28 €
Fiabilité = 93% → Prix prédit = 15.28 €
Fiabilité = 94% → Prix prédit = 15.28 €
Fiabilité = 95% → Prix prédit = 15.28 €
Fiabilité = 96% → Prix prédit = 15.54 €
Fiabilité = 97% → Prix prédit = 15.54 €
Fiabilité = 98% → Prix prédit = 16.1 €
Fiabilité = 99% → Prix prédit = 16.1 €
Fiabilité = 100% → Prix prédit = 15.98 €


In [3]:
df = pd.read_csv("data/fiverr_cleaned.csv")
print(df["Fiabilite"].describe())
print(df[["Fiabilite", "Prix"]].corr())

count    1252.000000
mean        0.739307
std         0.125703
min         0.081877
25%         0.698787
50%         0.789447
75%         0.826738
max         0.873558
Name: Fiabilite, dtype: float64
           Fiabilite      Prix
Fiabilite   1.000000 -0.018182
Prix       -0.018182  1.000000
